![Inteligência Artificial: Buscas em Textos com Python](http://iaexpert.com.br/wp-content/uploads/2018/01/300x141-LOGO-CURSO-SITE.png)
# Crawler e indexador de documentos
Estudo baseado nas aulas do curso: [Inteligência Artificial: Buscas em Textos com Python](https://www.udemy.com/inteligencia-artificial-buscas-em-textos-com-python/?couponCode=BTPSITE1) do professor Jones Granatyr na [IAExpert](http://iaexpert.com.br/).

Outros cursos da IAExpert: [IAExpert/cursos](http://iaexpert.com.br/index.php/cursos/)

In [1]:
# imports realizados a partir de pip install -r requeriments.txt
# nltk_data carregado com nltk.download('all')
import urllib3
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import re
import nltk
import pymysql

## Conexão com o banco de dados
Sql com as tabelas utilizadas no banco de dados de teste: [tabelas.sql](tabelas.sql)

In [2]:
DB = 'crawler'
def conecta_db():
    try:
        conexao = pymysql.connect(
            host='localhost',
            user='root',
            passwd='senhaDoMySql',
            db=DB,
            autocommit = True,
            use_unicode = True, 
            charset = 'utf8mb4')
        return conexao
    except Exception as e:
        print('Erro ao conectar no Banco de Dados', e)
        return False

conexao = conecta_db()

In [3]:
conexao.open

True

In [4]:
conexao.close()

In [5]:
conexao.open

False

## insere_registro()
Função "genérica" para inserir um novo registro numa tabela do banco de dados. Retorna o id do registro inserido

In [6]:
def insere_registro(tabela, colunas, valores):
    conexao = conecta_db()
    cursor = conexao.cursor()
    if isinstance(valores, str):
        valores = [valores]
    values = ''
    for valor in valores:
        values += ', \'' + str(valor) + '\''
    values = values.strip(', ')
    sql = 'insert into {} ({}) values({})'.format(tabela, colunas, values)
    try:
        cursor.execute(sql)
        ultimo = cursor.lastrowid
        cursor.close()
        conexao.close()
        return ultimo
    except Exception as e:
        print('Erro ao gravar o registro', e)
        return False
    

In [7]:
# teste que insere a palavra mundo na tabela palavras
insere_registro('palavras', 'palavra' ,'mundo')

1

In [8]:
# teste com erro, tabela não existe
insere_registro('palavrax', 'palavra' ,'mundo')

Erro ao gravar o registro (1146, "Table 'crawler.palavrax' doesn't exist")


False

## Funções para inserção de registros
Utilizam `insere_registro()` para gravar no Banco de dados

In [9]:
# Função para inserir uma nova palavra na tabela palavras, retornando seu id
def inserePalavra(palavra):
    tabela = 'palavras'
    coluna = 'palavra'
    return insere_registro(tabela, coluna, palavra)

In [10]:
inserePalavra('olá')

2

In [11]:
inserePalavra('teste')

3

In [12]:
# Função para inserir uma nova url na tabela urls, retornando seu id
def inserePagina(url):
    tabela = 'urls'
    coluna = 'url'
    cursor = conexao.cursor()
    return insere_registro(tabela, coluna, url)

In [13]:
inserePagina('teste2.com.br')

1

In [14]:
inserePagina('teste5.com')

2

In [15]:
inserePagina('maisumapagina.com')

3

In [16]:
# Função para gravar a localização de uma palavra em uma url
def inserePalavraLocalizacao(idurl, idpalavra, localizacao):
    tabela = 'palavra_localizacao'
    colunas = 'idurl, idpalavra, localizacao'
    values = [idurl, idpalavra, localizacao]
    return insere_registro(tabela, colunas, values)

In [17]:
inserePalavraLocalizacao('1', '2', '10')

1

In [18]:
inserePalavraLocalizacao(2, 1, 40)

2

In [19]:
# Função para gravar a ligação entre duas urls para uso no PageRank
def insertUrlLigacao(idurl_origem, idurl_destino):
    tabela = 'url_ligacao'
    colunas = 'idurl_origem, idurl_destino'
    values = [idurl_origem, idurl_destino]
    return insere_registro(tabela, colunas, values)

In [20]:
insertUrlLigacao('1', '3')

1

In [21]:
insertUrlLigacao(2, 3)

2

In [22]:
# Função para gravar a ligação entre uma palavra e o texto do link
def insertUrlPalavra(idpalavra, idurl_ligacao):
    tabela = 'url_palavra'
    colunas = 'idpalavra, idurl_ligacao'
    values = [idpalavra, idurl_ligacao]
    return insere_registro(tabela, colunas, values)

In [23]:
insertUrlPalavra(3, 1)

1

## procura_registro()
Função "genérica" para busca de um registro em uma tabela do Banco de Dados.

In [24]:
def procura_registro(tabela, campos, condicao):
    retorno = []
    conexao = conecta_db()
    cursor = conexao.cursor()
    sql = 'select {} from {} where {}'.format(campos, tabela, condicao)
    try:
        cursor.execute(sql)
        if cursor.rowcount > 0:
            retorno = cursor.fetchone()
        cursor.close()
        conexao.close()
        return retorno
    except Exception as e:
        print('Erro na busca do registro', e)
        return False
        

In [25]:
procura_registro('palavras', 'idpalavra', "palavra = 'olá'")

(2,)

In [26]:
procura_registro('palavras', '*', "palavra = 'olá'")

(2, 'olá')

In [27]:
# busca com erro no nome da tabela
procura_registro('palavrax', '*', "palavra = 'olá'")

Erro na busca do registro (1146, "Table 'crawler.palavrax' doesn't exist")


False

In [28]:
# busca com erro na condição
procura_registro('palavras', '*', "palavra = olá")

Erro na busca do registro (1054, "Unknown column 'olá' in 'where clause'")


False

## Funções para busca de registros
Utilizam `procura_registro()` para fazer a busca

In [29]:
# retorna o Id da url
def getIdUrl(url):
    idurl = -1
    tabela = 'urls'
    campo = 'idurl'
    condicao = "url = '{}'".format(url)
    result = procura_registro(tabela, campo, condicao)
    if result:
        idurl = result[0]
    return idurl

getIdUrl('teste5.com')

2

In [30]:
getIdUrl('xxxxx.com')

-1

In [31]:
# retorna o id da ligacação entre duas urls, origem e destino
def getIdUrlLigacao(idurl_origem, idurl_destino):
    idurl_ligacao = -1
    tabela = 'url_ligacao'
    campo = 'idurl_ligacao'
    condicao = "idurl_origem = '{}' and idurl_destino = '{}'".format(idurl_origem, idurl_destino)
    result = procura_registro(tabela, campo, condicao)
    if result:
        idurl_ligacao = result[0]
    return idurl_ligacao

In [32]:
getIdUrlLigacao('2', '3')

2

In [33]:
getIdUrlLigacao(2, 3)

2

In [34]:
# verifica se a palavra já existe na tabela palavras
def palavraIndexada(palavra):
    idPalavra = -1 # não existe a palavra no índice
    tabela = 'palavras'
    campo = 'idpalavra'
    condicao = "palavra = '{}'".format(palavra)
    result = procura_registro(tabela, campo, condicao)
    if result:
        idPalavra = result[0]
    return idPalavra

In [35]:
palavraIndexada('olá')

2

In [36]:
palavraIndexada('olásss')

-1

### Verifica se a página já foi indexada
Retornos de `paginaIndexada()`:
* -1 -> se a página não existe
* -2 -> se página existe e já possui palavras indexadas
* id -> se ela existe, mas está pendente de indexação

In [37]:
def paginaIndexada(url):
    tabela = 'urls'
    campo = 'idurl'
    condicao = "url = '{}'".format(url)
    result = procura_registro(tabela, campo, condicao)
    if not result:
        return -1  # não existe a página
    idurl = result[0]
    tabela = 'palavra_localizacao'
    campo = 'idurl'
    condicao = "idurl = '{}'".format(idurl)
    result = procura_registro(tabela, campo, condicao)
    if result:
        return -2  # existe a página com palavras cadastradas
    return idurl  # existe a página sem palavras, então retorna o ID da página

In [38]:
# pagina não existe
paginaIndexada('teste')

-1

In [39]:
# pagina existe mas não tem palavras indexadas
paginaIndexada('maisumapagina.com')

3

In [40]:
# pagina existe e tem palavras indexadas
paginaIndexada('teste5.com')

-2

## Funções utilizadas pelo Indexador

In [43]:
# transforma o texto numa lista de palavras padronizadas
def separaPalavras(texto):
    stop = nltk.corpus.stopwords.words('portuguese')
    stemmer = nltk.stem.RSLPStemmer()
    splitter = re.compile('\\W+')
    lista_palavras = []
    lista = [p for p in splitter.split(texto) if p != '']
    for p in lista:
        if p.lower() in stop or len(p) <= 1:
            continue
        lista_palavras.append(stemmer.stem(p).lower())
    return lista_palavras

link = 'https://pt.wikipedia.org/wiki/Linguagem_de_programa%C3%A7%C3%A3o'
link = link.replace('_', ' ')
palavras = separaPalavras(link)
palavras

['http',
 'pt',
 'wikiped',
 'org',
 'wik',
 'lingu',
 'program',
 'c3',
 'a7',
 'c3',
 'a3o']

In [44]:
# Faz a ligação entre uma url de origem e as palavras existentes no texto
# de uma url de destino indicada
def urlLigaPalavra(url_origem, url_destino):
    texto_url = url_destino.replace('_', ' ')
    palavras = separaPalavras(texto_url)
    idurl_origem = getIdUrl(url_origem)
    idurl_destino = getIdUrl(url_destino)
    if idurl_destino == -1:
        idurl_destino = inserePagina(url_destino)
    if idurl_origem == idurl_destino:
        return
    if getIdUrlLigacao(idurl_origem, idurl_destino) > 0:
        return
    idurl_ligacao = insertUrlLigacao(idurl_origem, idurl_destino)
    for palavra in palavras:
        idpalavra = palavraIndexada(palavra)
        if idpalavra == -1:
            idpalavra = inserePalavra(palavra)
        insertUrlPalavra(idpalavra, idurl_ligacao)

In [45]:
destino = 'pagina_de_destino.com'
urlLigaPalavra('teste5.com', destino)

In [46]:
idOrigem = getIdUrl('teste5.com')
idDestino = getIdUrl('pagina_de_destino.com')
idPalavra = palavraIndexada('destin')
idUrlLig = getIdUrlLigacao(idOrigem, idDestino)
registro_gravado = procura_registro('url_palavra', '*', "idPalavra='{}'".format(idPalavra))

In [47]:
print(idOrigem, idDestino)
print(idUrlLig)
print(idPalavra, idUrlLig)
print(registro_gravado)

2 4
3
5 3
(3, 5, 3)


In [48]:
def getTexto(sopa):
    for tags in sopa(['script', 'style']):
        tags.decompose()
    return ' '.join(sopa.stripped_strings)

In [49]:
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
http = urllib3.PoolManager()
link = 'https://ovoodaserpenteemplumada.com'
pagina = http.request('GET', link)
sopa = BeautifulSoup(pagina.data, "lxml")
getTexto(sopa)

'Apresentação | O Voo da Serpente Emplumada Navegação Apresentação Apresentação Livro Um Livro Dois Livro Três Baixe o Livro Glossário A Tradução O Voo da Serpente Emplumada Tradução do livro "El Vuelo de la Serpiente Emplumada" de Armando Cosani Nota: O desenvolvimento deste site, bem como a tradução do livro "O Voo da Serpente Emplumada" fazem parte de um projeto pessoal e sem fins lucrativos, cujo único objetivo é a difusão das belas e profundas mensagens transmitidas por este importante livro do Esoterismo Crístico de nossa época. Apresentação Envolta na trama de um relato que quase é um diálogo entre o narrador e um homem inexplicável — “todo ele era um sorriso” — que em palavras simples repete verdades eternas, vaga a presença de Judas, o homem de Kariot; na invocação à Santa Terra Bendita do Mayab, à Sagrada Princesa Sac-Nicté, a branca flor do Mayab e ao Grande Senhor Oculto, evoca-se o nome de Judas, o homem de Kariot. Porém, por que Judas? Não foi quem enlodou sua memória com

In [50]:
def indexador(url, sopa):
    indexada = paginaIndexada(url)
    if indexada == -2:
        print("url {} já indexada".format(url))
        return
    elif indexada == -1:
        idnova_pagina = inserePagina(url)
    elif indexada > 0:
        idnova_pagina = indexada
    print("Aguarde! Indexando " + url)
    texto = getTexto(sopa)
    palavras = separaPalavras(texto)
    for i in range(len(palavras)):
        palavra = palavras[i]
        idpalavra = palavraIndexada(palavra)
        if idpalavra == -1:
            idpalavra = inserePalavra(palavra)
        inserePalavraLocalizacao(idnova_pagina, idpalavra, i)
    print('Ok! ' + url + " indexada")

In [51]:
indexador(link, sopa)

Aguarde! Indexando https://ovoodaserpenteemplumada.com
Ok! https://ovoodaserpenteemplumada.com indexada


In [52]:
def crawl(paginas, profundidade, dominio=''):
    urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
    for i in range(profundidade):
        novas_paginas = set()
        for pagina in paginas:
            http = urllib3.PoolManager()
            try:
                dados_pagina = http.request('GET', pagina)
            except:
                print('Erro ao abrir a página ' + pagina)
                continue
            sopa = BeautifulSoup(dados_pagina.data, "lxml")
            indexador(pagina, sopa)
            links = sopa.find_all('a')
            contador = 1
            for link in links:
                #print(str(link.contents) + " - " + str(link.get('href')))
                #print(link.attrs)
                #print('\n')
                if not('href' in link.attrs):
                    continue
                url = urljoin(pagina, str(link.get('href')))
                if dominio:
                    if not(dominio in url):
                        continue
                if url.find("'") != -1:
                    continue
                #print(url)
                url = url.split('#')[0]
                #print(url)
                #print('\n')
                if url[0:4] == 'http':
                    novas_paginas.add(url) 
                urlLigaPalavra(pagina, url)
                contador = contador + 1
            paginas = novas_paginas
        print(contador)
    print('Indexação Concluída')

In [53]:
lista_paginas = ['https://ovoodaserpenteemplumada.com']
# para a indexação ficar dentro do domínio, numa aplicação onde não será aplicado o PageRank
# acrescentei um parâmetro que limita a indexação
crawl(lista_paginas, 3, 'ovoodaserpenteemplumada.com')

url https://ovoodaserpenteemplumada.com já indexada
20
Aguarde! Indexando https://ovoodaserpenteemplumada.com/livro-um/
Ok! https://ovoodaserpenteemplumada.com/livro-um/ indexada
Aguarde! Indexando https://ovoodaserpenteemplumada.com/livro-tres/
Ok! https://ovoodaserpenteemplumada.com/livro-tres/ indexada
Aguarde! Indexando https://ovoodaserpenteemplumada.com/cdn-cgi/l/email-protection
Ok! https://ovoodaserpenteemplumada.com/cdn-cgi/l/email-protection indexada
Aguarde! Indexando https://ovoodaserpenteemplumada.com/glossario/
Ok! https://ovoodaserpenteemplumada.com/glossario/ indexada
Aguarde! Indexando https://ovoodaserpenteemplumada.com/sobre-a-traducao/
Ok! https://ovoodaserpenteemplumada.com/sobre-a-traducao/ indexada
Aguarde! Indexando https://ovoodaserpenteemplumada.com/livro-dois/
Ok! https://ovoodaserpenteemplumada.com/livro-dois/ indexada
Aguarde! Indexando https://ovoodaserpenteemplumada.com/livro-um
Ok! https://ovoodaserpenteemplumada.com/livro-um indexada
Aguarde! Indexando 

Ok! https://ovoodaserpenteemplumada.com/livro-um/livro-um-capitulo-onze/ indexada
url https://ovoodaserpenteemplumada.com/livro-tres/ já indexada
Aguarde! Indexando https://ovoodaserpenteemplumada.com/livro-dois/livro-dois-capitulo-cinco/
Ok! https://ovoodaserpenteemplumada.com/livro-dois/livro-dois-capitulo-cinco/ indexada
Aguarde! Indexando https://ovoodaserpenteemplumada.com/livro-um/livro-um-capitulo-quinze
Ok! https://ovoodaserpenteemplumada.com/livro-um/livro-um-capitulo-quinze indexada
Aguarde! Indexando https://ovoodaserpenteemplumada.com/livro-tres/livro-tres-capitulo-oito/
Ok! https://ovoodaserpenteemplumada.com/livro-tres/livro-tres-capitulo-oito/ indexada
url https://ovoodaserpenteemplumada.com/glossario/ já indexada
Aguarde! Indexando https://ovoodaserpenteemplumada.com/livro-um/livro-um-capitulo-seis/
Ok! https://ovoodaserpenteemplumada.com/livro-um/livro-um-capitulo-seis/ indexada
Aguarde! Indexando https://ovoodaserpenteemplumada.com/livro-um/livro-um-capitulo-cinco/
Ok